In [62]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch as th
# import torchmetrics as thm
import LPGNN
import igraph as ig
import networkx as nx
import torch_geometric as pyg

import pyarrow as pa
import pyarrow.parquet as pq

import importlib
import powerlaw

import imageio

from tqdm import tqdm

In [88]:
importlib.reload(LPGNN.popularity_similarity)
importlib.reload(LPGNN.embedding)
importlib.reload(LPGNN.poincare_embedding)
importlib.reload(LPGNN.distances)
importlib.reload(LPGNN.visualization.degree_distribution)
importlib.reload(LPGNN.visualization)
importlib.reload(LPGNN.utils)
importlib.reload(LPGNN.labne)
importlib.reload(LPGNN.graph_metrics)
importlib.reload(LPGNN.DataSetup)
importlib.reload(LPGNN.tree_pruning)
importlib.reload(LPGNN.LinkPrediction)

<module 'LPGNN.LinkPrediction' from '/home/mf/Documents/IB/Tesis/Maestria/LPGNN/LPGNN/LinkPrediction.py'>

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


In [63]:
th.set_default_tensor_type(th.DoubleTensor)

In [64]:
# generate a tensor of 20 points in dimension N=3, with a radius of 1
N = 3
n_points = 5
r = th.Tensor(n_points).uniform_(0, 1)
angulars = th.Tensor(n_points, N-1).uniform_(0, 2*np.pi)
x = th.cat([r.unsqueeze(1), angulars], dim=1)
x = LPGNN.distances.to_cartesian(x)

In [115]:
x[0][0] = .9999999
x[0][1] = 0.0
x[0][2] = 0.0

x[1][0] = -0.999999978
x[1][1] = 0.0
x[1][2] = 0.0

In [116]:
def poincare_dist(u:th.Tensor, v:th.Tensor):
    """ Compute the Poincare distance between two vectors. """
    sqdist = th.sum((u - v) ** 2, dim=-1) 
    squnorm = th.sum(u ** 2, dim=-1)
    sqvnorm = th.sum(v ** 2, dim=-1) 
    d = 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm) + sqdist)
    return 2 * th.log( (th.sqrt(sqdist) + th.sqrt(squnorm*sqvnorm - 2*th.dot(u,v) + 1)) / th.sqrt((1-squnorm)*(1-sqvnorm)) )
    #return 2 * th.arcsinh(d/2)
    x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + 1e-15
    z = th.sqrt(x ** 2 - 1)
    return th.log(x + z)
    # sqdist = max_r**2 * th.sum((u - v) ** 2, dim=-1)
    # squnorm = max_r**2 - th.sum(u ** 2, dim=-1)
    # sqvnorm = max_r**2 - th.sum(v ** 2, dim=-1)

In [117]:
for i in range(n_points):
    for j in range(i):
        print('        ', end='')
    for j in range(i+1, n_points):
        t = LPGNN.distances.poincare_dist(x[i], x[j])
        print('%07.4f'%t.item(), end=' ')
    print(' ')

35.1366 17.3176 18.4515 18.6089  
        18.5016 16.8820 16.7013  
                01.6929 02.0493  
                        00.7662  
                                 


In [118]:
for i in range(n_points):
    for j in range(i+1, n_points):
        t = LPGNN.distances.poincare_dist(x[i], x[j])
        t_ = poincare_dist(x[i], x[j])
        print(i,j,'%07.4f'%t.item(), '%07.4f'%t_.item())

0 1 35.1366 35.1366
0 2 17.3176 17.3176
0 3 18.4515 18.4515
0 4 18.6089 18.6089
1 2 18.5016 18.5016
1 3 16.8820 16.8820
1 4 16.7013 16.7013
2 3 01.6929 01.6929
2 4 02.0493 02.0493
3 4 00.7662 00.7662


In [201]:
LPGNN.distances.poincare_dist(x[0], x[1])

tensor(4.)
tensor(1.0000)
tensor(1.)
tensor(inf)
tensor(inf)


tensor(inf)

In [197]:
np.log(2*5.6295e150)

347.80893175792784

In [12]:
x = x.float

In [19]:
x = x.double()

In [26]:
x[0][0] = 1.12345678910111213141516

In [27]:
x[0][0].item()

1.1234567891011122

In [60]:
x = th.tensor(1., requires_grad=True)
y = th.tensor(2., requires_grad=True)
z = th.tensor(3., requires_grad=True)

f = x**2 + y**3 + th.cos(z)
g = th.sin(x)
h = f*g

# calculate partial derivatives
df_dx = th.autograd.grad(f, x, create_graph=True)[0]

tensor(2., grad_fn=<MulBackward0>)

In [57]:
h.backward()

In [58]:
h

tensor(6.7402, grad_fn=<MulBackward0>)

Since $f = x^2 + y^2$, $g = \cos(z)$ and $h = fg$, then using backpropagation we have

$$ \frac{dh}{dx} = \frac{dh}{df} \frac{df}{dx} + \frac{dh}{dg} \frac{dg}{dx} = 2x + 0 = 2x$$

$$ \frac{dh}{dy} = \frac{dh}{df} \frac{df}{dy} + \frac{dh}{dg} \frac{dg}{dy} = 3y^2 + 0 = 3y^2$$

$$ \frac{dh}{dz} = \frac{dh}{df} \frac{df}{dz} + \frac{dh}{dg} \frac{dg}{dz} = 0 + -\sin(z) = -\sin(z)$$

In [59]:
x.grad, y.grad, z.grad

(tensor(6.0108), tensor(10.0977), tensor(-0.1187))

(tensor(12., grad_fn=<MulBackward0>),
 tensor(2., grad_fn=<MulBackward0>),
 tensor(-0.1411, grad_fn=<NegBackward0>))